## Description of Program
- program:    REFUSA_1av1_ObtainReferenceUSA
- task:       Read in large Historic Reference USA data
- Version:    2021-08-27
- project:    Interdependent Networked Community Resilience Modeling Environment (IN-CORE) Subtask 5.2 - Social Institutions
- funding:	  NIST Financial Assistance Award Numbers: 70NANB15H044 and 70NANB20H008 
- author:     Nathanael Rosenheim

- Suggested Citation:

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Display versions being used - important information for replication
import sys
print("Python Version     ", sys.version)
print("pandas version:    ", pd.__version__)
print("numpy version:    ", np.__version__)

Python Version      3.8.12 | packaged by conda-forge | (default, Oct 12 2021, 21:22:46) [MSC v.1916 64 bit (AMD64)]
pandas version:     1.3.5
numpy version:     1.22.0


In [3]:
os.getcwd()

'g:\\Shared drives\\HRRC_IN-CORE\\Tasks\\M5.2-01 Pop inventory\\github_com\\npr99\\Population_Inventory\\pyincore_data_addons\\SourceData\\www_data_axle_com'

In [4]:
# to access new package that is in a sibling folder - the system path list needs to inlcude the parent folder (..)
# append the path of the directory that includes the github repository.
sys.path.append("..\\github_com\\npr99\\Population_Inventory")

In [5]:
# To reload submodules need to use this magic command to set autoreload on
%load_ext autoreload
%autoreload 2
from pyincore_data_addons.SourceData.www_data_axle_com.dataaxle_01a_obtain \
    import obtain_dataaxel

ModuleNotFoundError: No module named 'pyincore_data_addons'

In [18]:
import wget
year = '2010'
url = 'C:\\MyProjects\\HRRCProjects\\IN-CORE\\WorkNPR\\'
filename = f"{url}BusinessData{year}.csv.zip"

chunk = pd.read_csv(filename, encoding='ISO-8859-1',  chunksize=1000000, 
    compression='zip', header=0, sep=',', quotechar='"', low_memory=False)
pd_df = pd.concat(chunk)

In [20]:
pd_df.head(1).T

,0
Company,QUINCY COMPRESSOR
Address_Line_1,701 N DOBSON AVE
City,BAY MINETTE
State,AL
ZipCode,36507.0
Zip4,3199.0
County_Code,3.0
Area_Code,251
IDCode,2
Location_Employee_Size_Code,F


In [21]:
FIPS_Code = 37155
State = "NC"
select_pd_df = pd_df.loc[(pd_df['FIPS_Code'] == FIPS_Code) &
                        (pd_df['State'] == State)].copy(deep=True)

In [22]:
# Create 2-digit NAICS Code
select_pd_df.loc[:,'Primary_NAICS_Code'] = \
    select_pd_df['Primary_NAICS_Code'].fillna(00000)
select_pd_df['NAICS2D'] = select_pd_df['Primary_NAICS_Code'].\
    apply(lambda x : str(int(x))[0:2])
# Add 4-digit NAICS Codes for more detail
# Example Education Services: NAICS 61
#  Elementary and Secondary Schools: NAICS 6111 
select_pd_df['NAICS4D'] = select_pd_df['Primary_NAICS_Code'].\
    apply(lambda x : str(int(x))[0:4])

In [24]:
# Keep Company Name if Education or Health Care Services
# Social Institution Name
select_pd_df['SIName'] = ""
education_services = (select_pd_df['NAICS2D'] == '61')
select_pd_df.loc[education_services, 'SIName'] = select_pd_df['Company']
health_care = (select_pd_df['NAICS2D'] == '62')
select_pd_df.loc[health_care, 'SIName'] = select_pd_df['Company']

In [27]:
def add_estabid(add_estabid_df):
    add_estabid_df.loc[:,'FIPS_Code_str'] = \
        add_estabid_df['FIPS_Code'].apply(lambda x : str(int(x)).zfill(5))
    add_estabid_df.loc[:,'estabid_part1'] = "E"+add_estabid_df['FIPS_Code_str'] + \
        add_estabid_df['NAICS2D'].apply(lambda x : str(int(x)).zfill(2))
    # Part2 of unique id is a counter based on the part1
    # The counter ensures that values are unique within part1
    # Add unique ID based on group vars
    add_estabid_df.loc[:,'estabid_part2'] = \
        add_estabid_df.groupby(['estabid_part1']).cumcount() + 1
    # Need to zero pad part2 find the max number of characters
    part2_max = add_estabid_df['estabid_part2'].max()
    part2_maxdigits = len(str(part2_max))
    add_estabid_df.loc[:,'estabid'] = add_estabid_df['estabid_part1'] + \
        add_estabid_df['estabid_part2'].apply(lambda x : \
            str(int(x)).zfill(part2_maxdigits))

    return add_estabid_df



In [28]:
# Add Unique Establishment ID
select_pd_df = add_estabid(select_pd_df)

In [29]:
# Only save a portion of the original file
keep_vars = ['estabid','NAICS2D','NAICS4D','SIName',
    'Latitude','Longitude','Employee_Size_Location']

In [30]:
select_pd_df[keep_vars]

,estabid,NAICS2D,NAICS4D,SIName,Latitude,Longitude,Employee_Size_Location
2773,E3715533001,33,3329,,34.819838,-79.001781,225.0
3050,E3715531001,31,3159,,34.589000,-79.002420,125.0
3051,E3715531002,31,3121,,34.614744,-79.000010,66.0
6965,E3715532001,32,3222,,34.640002,-79.073161,120.0
8969,E3715562001,62,6221,SOUTHEASTERN REGIONAL MED CTR,34.637220,-79.012130,1900.0
...,...,...,...,...,...,...,...
13658235,E3715592235,92,9221,,34.844004,-79.107316,26.0
13658236,E3715544511,44,4451,,34.832130,-79.083680,4.0
13658237,E3715544512,44,4413,,34.844004,-79.107316,3.0
13823483,E3715556069,56,5617,,34.754840,-78.882860,3.0


In [23]:
select_pd_df.groupby('NAICS2D').\
        aggregate({'Employee_Size_Location':np.sum})

,Employee_Size_Location
NAICS2D,
0,7.0
11,889.0
21,9.0
22,181.0
23,2284.0
31,2715.0
32,784.0
33,1575.0
42,920.0


In [3]:
import os

# Store Program Name for output files to have the same name
programname = "REFUSA_1av1_ObtainReferenceUSA"

# Save Outputfolder - due to long folder name paths output saved to folder with shorter name
# files from this program will be saved with the program name - this helps to follow the overall workflow
outputfolder = "workflow_output"
# Make directory to save output
if not os.path.exists(outputfolder):
    os.mkdir(outputfolder)

## Background on Data

https://platform.data-axle.com/places/attributes

### Other universities with data access

https://lib.msu.edu/about/data/referenceusahistorical/

Infogroup, 2014, "ReferenceUSA Business Historical Data Files", https://doi.org/10.7910/DVN/GW2P3G, Harvard Dataverse, V13

### ReferenceUSA Historical Business (1997-2020)
The ReferenceUSA Historical Business data collection provides access to company profiles from 1997 through 2020. Variables include company name, address, SIC/NAICS codes, employee size, sales, latitude/longitude, and more. ReferenceUSA is a product of Data Axle, formerly Infogroup.

### Access
Access to ReferenceUSA Historical Business data is limited to current TAMU affiliates only and requires login with your TAMU netID and password. Redistribution is not permitted.

### Guidelines for Acceptable Use
Prohibition of commercial use, and guidelines for acceptable use

The data is only for academic or private study in compliance with applicable laws. You are strictly prohibited from redistributing or reselling of the data. You may publish limited excerpts and a summarized or aggregated statistics or analysis of the data in academic and scholarly works but not the data in raw form.

BY CONTINUING TO USE THIS DATA COLLECTION YOU ARE AGREEING TO ACCEPT THESE TERMS.

## Guidelines described well on MSU website
I have replaced the MSU with TAMU - seems to apply no matter what the university is.
https://libguides.lib.msu.edu/c.php?g=95386&p=623717.

### Notice to users of licensed databases
### Prohibition of commercial use, and guidelines for acceptable use
The [TAMU] Libraries subscribe to licensed database products to support the educational and research needs of library users. In some cases, these databases are educational versions of commercial products. Users are advised that access to these materials is controlled by license agreements: violation of license terms by individual library users potentially jeopardizes future campus access for all students and faculty, and exposes violaters to sanctions.

The content of these databases are made available only for the individual educational and research purposes of authorized users. By proceeding to the database itself, you as the user are indicating that you are aware of the following terms and conditions, and agree to conduct your use of this material accordingly.

#### Uses that are allowed:
- You may use the database for purposes of academic research or private study only.
- You may browse and search the database, and display its contents on the screen.
- You may make and save a digital copy of limited extracts from the database for academic purposes.
- You may print out copies of limited extracts from the database for academic purposes.
- You may reproduce or quote limited portions of the database contents for reports, essays, projects, and similar materials created for academic purposes, with appropriate acknowledgement of the source (such as footnotes, endnotes or other citations).
- These limited extracts may be shared with other academic users.

#### Uses that are NOT allowed:
- You may not sell or otherwise re-distribute data to third parties without express permission.This includes but is not limited to posting on public sources like Google Docs, Tableau, etc.
- You may not use the database or any part of the information comprised in the database content for commercial research, for example, research that is done under a funding or consultant contract, internship, or other relationship in which the results are delivered to a for-profit organization.
- You may not engage in bulk reproduction or distribution of the licensed materials in any form.
- You may not engage in extensive downloading or copying of content.
- You may not use automated searching or querying, including, but not limited to, the use of spiders or other external software for text and data mining.
- You may not store a vast amount of data on your personal computers.

In [4]:
filename = "C:/Users/nathanael99/MyProjects/IN-CORE/SourceData/dataaxle/BusinessData2016.csv"

In [5]:
# Check encoding
# https://stackoverflow.com/questions/37177069/how-to-check-encoding-of-a-csv-file/52648848
with open(filename) as f:
    print(f)

<_io.TextIOWrapper name='C:/Users/nathanael99/MyProjects/IN-CORE/SourceData/dataaxle/BusinessData2016.csv' mode='r' encoding='cp1252'>


In [6]:
refusa_df = pd.read_csv(filename, nrows = 1000, encoding='cp1252')
# look at 1 example
refusa_df.head(1).T

,0
Company,QUINCY COMPRESSOR
Address_Line_1,701 N DOBSON AVE
City,BAY MINETTE
State,AL
ZipCode,36507
Zip4,3199.0
County_Code,3
Area_Code,251
IDCode,2
Location_Employee_Size_Code,F


In [7]:
for col in refusa_df:
    print(col)

Company
Address_Line_1
City
State
ZipCode
Zip4
County_Code
Area_Code
IDCode
Location_Employee_Size_Code
Location_Sales_Volume_Code
Primary_SIC_Code
SIC6_Descriptions
Primary_NAICS_Code
NAICS8_Descriptions
SIC_Code
SIC6_Descriptions_SIC
SIC_Code_1
SIC6_Descriptions_SIC1
SIC_Code_2
SIC6_Descriptions_SIC2
SIC_Code_3
SIC6_Descriptions_SIC3
SIC_Code_4
SIC6_Descriptions_SIC4
Archive_Version_Year
Yellow_Page_Code
Employee_Size_Location
Sales_Volume_Location
Business_Status_Code
Industry_Specific_First_Byte
Year_Established
Office_Size_Code
Company_Holding_Status
ABI
Subsidiary_Number
Parent_Number
Parent_Actual_Employee_Size
Parent_Actual_Sales_Volume
Parent_Employee_Size_Code
Parent_Sales_Volume_Code
Site_Number
Address_Type_Indicator
Population_Code
Census_Tract
Census_Block
Latitude
Longitude
Match_Code
CBSA_Code
CBSA_Level
CSA_Code
FIPS_Code
franchise


### ISSUE NOTE 
File does not include the IUSA Number - which is the unique id for the buisnesses in the data downloaded in the past. Need to see if this is true for other years.

Need the IUSA number to tract businesses over time.

Help with reading in large CSV files:
https://medium.com/analytics-vidhya/optimized-ways-to-read-large-csvs-in-python-ab2b36a7914e

help from https://stackoverflow.com/questions/28239529/conditional-row-read-of-csv-in-pandas

## Attempt to read file

This is proving to be very problematic. The file is so large and probably has some issues with the comma seperators.

Various errors:

UnicodeDecodeError: 'charmap' codec can't decode byte 0x81 in position 96936: character maps to <undefined>

UnicodeDecodeError: 'charmap' codec can't decode byte 0x81 in position 77953704: character maps to <undefined>

UnicodeDecodeError: 'charmap' codec can't decode byte 0x81 in position 137560: character maps to <undefined>

In [8]:
print("Attempting to read in ", filename)
# attempt1_df = pd.read_csv(filename)
print("Attempt 1 errror: UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe0 in position 123778: invalid continuation byte")

Attempting to read in  C:/Users/nathanael99/MyProjects/IN-CORE/SourceData/dataaxle/BusinessData2016.csv
Attempt 1 errror: UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe0 in position 123778: invalid continuation byte


In [9]:
print("Attempt 2 to read in using encoding cp1252", filename)
# attempt2_df = pd.read_csv(filename, encoding='cp1252')
print("Attempt 2 errror: UnicodeDecodeError: 'charmap' codec can't decode byte 0x81 in position 137560: character maps to <undefined>")

Attempt 2 to read in using encoding C:/Users/nathanael99/MyProjects/IN-CORE/SourceData/dataaxle/BusinessData2016.csv
Attempt 2 errror: UnicodeDecodeError: 'charmap' codec can't decode byte 0x81 in position 137560: character maps to <undefined>


### Found correct encoding
Opened data in Stata and found that the encoding was ISO-8859-1.

Also learned that python has issues with decoding some characters - need to check the final data to make sure it looks correct.

In [12]:
print("Attempt 3 to read in using encoding ISO-8859-1", filename)
attempt2_df = pd.read_csv(filename, encoding='ISO-8859-1')

Attempt 3 to read in using encoding ISO-8859-1 C:/Users/nathanael99/MyProjects/IN-CORE/SourceData/dataaxle/BusinessData2016.csv


C:\Users\nathanael99\Miniconda3\envs\pyincoreEnv20210802\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (18,20,22,24,30,39,40) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [13]:
chunk = pd.read_csv(filename,chunksize=1000000, encoding='ISO-8859-1', low_memory=False)
pd_df = pd.concat(chunk)

In [45]:
#select_pd_df = pd_df.loc[pd_df['FIPS_Code'] == 48167].copy(deep=True)
select_pd_df = pd_df.loc[(pd_df['City'] == 'LUMBERTON') &
                         (pd_df['State'] == 'NC')].copy(deep=True)

In [46]:
select_pd_df[['FIPS_Code','Parent_Number','City','State']].astype(str).describe()

,FIPS_Code,Parent_Number,City,State
count,2547,2547,2547,2547
unique,2,226,1,1
top,37155.0,nan,LUMBERTON,NC
freq,2546,2151,2547,2547


In [47]:
select_pd_df.loc[:,'Primary_NAICS_Code'] = select_pd_df['Primary_NAICS_Code'].fillna(00000)
select_pd_df['NAICS2D'] = select_pd_df['Primary_NAICS_Code'].apply(lambda x : str(int(x))[0:2])

In [48]:
select_pd_df.groupby('NAICS2D').aggregate({'Employee_Size_Location':np.sum})

,Employee_Size_Location
NAICS2D,
0,16.0
11,70.0
22,36.0
23,1204.0
31,995.0
32,998.0
33,1140.0
42,588.0
44,2441.0


### Save output as CSV

In [49]:
# Save Work at this point as CSV
savefile = sys.path[0]+"/"+outputfolder+"/"+programname+"_2016_Lumberton.csv"
select_pd_df.to_csv(savefile)